# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut

In [10]:
task = 'Cow_labeling' # Enter the name of your experiment Task
experimenter = 'FKIE' # Enter the name of the experimenter
video = ['data/Tag1/cow1.mp4', 'data/Tag1/cow24.mp4'] # Enter the paths of your videos you want to grab frames from.

deeplabcut.create_new_project(task,experimenter,video, working_directory='/home/wei-chan/Documents/Thesis/src/annotation', copy_videos=True) #change the working directory to where you want the folders created.

Created "/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/videos"
Created "/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/labeled-data"
Created "/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/training-datasets"
Created "/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models"
Copying the videos
data/Tag1/cow1.mp4
data/Tag1/cow24.mp4
Generated "/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/config.yaml"

A new project with name Cow_labeling-FKIE-2019-01-02 is created at /home/wei-chan/Documents/Thesis/src/annotation and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames' to select frames for labeling.
. [OPTIONAL] Use the function 'add_new_videos' to add new videos to your project (at any stage).


'/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/config.yaml'

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [11]:
%matplotlib inline
path_config_file = '/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.extract_frames(path_config_file,'automatic','kmeans', crop=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#You can change the cropping to false, then delete the checkcropping part!

38it [00:00, 373.66it/s]

Reading config file successfully...
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 9.3  seconds.
Extracting and downsampling... 186  frames from the video.


186it [00:00, 363.99it/s]


Kmeans clustering ... (this might take a while)


37it [00:00, 364.09it/s]

Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 6.1  seconds.
Extracting and downsampling... 122  frames from the video.


122it [00:00, 343.43it/s]


Kmeans clustering ... (this might take a while)

Frames are selected.
You can now label the frames using the function 'label_frames'.


## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [2]:
path_config_file = '/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/config.yaml'

In [ ]:
%gui wx
deeplabcut.label_frames(path_config_file)

Working on folder: cow1
Working on folder: cow24
Working on folder: cow24


**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [4]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by FKIE.
They are stored in the following folder: /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/labeled-data/cow1_labeled.
They are stored in the following folder: /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/labeled-data/cow24_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [5]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training - If you want to use a CPU, continue. 
### If yu want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [7]:
deeplabcut.train_network(path_config_file, gputouse=0)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]],
 'all_joints_names': ['head',
                      'neck',
                      'withers',
                      'back1',
                      'back2',
                      'back3',
                      'hoofFL',
                      'hoofFR',
                      'hoofHL',
                      'hoofHR'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Cow_labelingJan2/Cow_labeling_FKIE90shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 500,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/wei-chan/anaconda3/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss

INFO:tensorflow:Restoring parameters from /home/wei-chan/anaconda3/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Restoring parameters from /home/wei-chan/anaconda3/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt
Restoring parameters from /home/wei-chan/anaconda3/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight': 400, 'all_joints': [[0], [1], [2], [3], [4], [5], [6]

iteration: 500 loss: 0.0399 lr: 0.005
iteration: 500 loss: 0.0399 lr: 0.005
iteration: 1000 loss: 0.0193 lr: 0.005
iteration: 1000 loss: 0.0193 lr: 0.005
iteration: 1500 loss: 0.0153 lr: 0.005
iteration: 1500 loss: 0.0153 lr: 0.005
iteration: 2000 loss: 0.0129 lr: 0.005
iteration: 2000 loss: 0.0129 lr: 0.005
iteration: 2500 loss: 0.0111 lr: 0.005
iteration: 2500 loss: 0.0111 lr: 0.005
iteration: 3000 loss: 0.0104 lr: 0.005
iteration: 3000 loss: 0.0104 lr: 0.005
iteration: 3500 loss: 0.0092 lr: 0.005
iteration: 3500 loss: 0.0092 lr: 0.005
iteration: 4000 loss: 0.0087 lr: 0.005
iteration: 4000 loss: 0.0087 lr: 0.005
iteration: 4500 loss: 0.0080 lr: 0.005
iteration: 4500 loss: 0.0080 lr: 0.005
iteration: 5000 loss: 0.0075 lr: 0.005
iteration: 5000 loss: 0.0075 lr: 0.005
iteration: 5500 loss: 0.0071 lr: 0.005
iteration: 5500 loss: 0.0071 lr: 0.005
iteration: 6000 loss: 0.0066 lr: 0.005
iteration: 6000 loss: 0.0066 lr: 0.005
iteration: 6500 loss: 0.0064 lr: 0.005
iteration: 6500 loss: 0.006

KeyboardInterrupt: 

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [8]:
deeplabcut.evaluate_network(path_config_file)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]],
 'all_joints_names': ['head',
                      'neck',
                      'withers',
                      'back1',
                      'back2',
                      'back3',
                      'hoofFL',
                      'hoofFR',
                      'hoofHL',
                      'hoofHR'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Cow_labelingJan2/Cow_labeling_FKIE90shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 500,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/wei-chan/anaconda3/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss

Running  DeepCut_resnet50_Cow_labelingJan2shuffle1_21000  with # of trainingiterations: 21000
INFO:tensorflow:Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-21000


Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-21000
Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-21000
0it [00:00, ?it/s]

Analyzing data...


38it [00:03, 12.29it/s]

Done and results stored for snapshot:  snapshot-21000
Results for 21000  training iterations: 90 1 train error: 2.69 pixels. Test error: 10.54  pixels.
With pcutoff of 0.1  train error: 2.69 pixels. Test error: 10.54 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [12]:
videofile_path = ['/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1.mp4','/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow24.mp4'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]],
 'all_joints_names': ['head',
                      'neck',
                      'withers',
                      'back1',
                      'back2',
                      'back3',
                      'hoofFL',
                      'hoofFR',
                      'hoofHL',
                      'hoofHR'],
 'batch_size': 4,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Cow_labelingJan2/Cow_labeling_FKIE90shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 500,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/wei-chan/anaconda3/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss

Using snapshot-21000 for model /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1
INFO:tensorflow:Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-21000


Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-21000
Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-21000


/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1.mp4
Starting %  /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1.mp4
Video already analyzed! /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1DeepCut_resnet50_Cow_labelingJan2shuffle1_21000.h5
/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow24.mp4
Starting %  /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow24.mp4
Video already analyzed! /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow24DeepCut_resnet50_Cow_labelingJan2shuffle1_21000.h5
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [7]:
deeplabcut.extract_outlier_frames?

In [8]:
deeplabcut.extract_outlier_frames(path_config_file,['/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1.mp4'], comparisonbodyparts = ['hoofFL','hoofFLR','hoofHL', 'hoofHR'], epsilon = 5, extractionalgorithm = 'kmeans')

  0%|          | 0/4 [00:00<?, ?it/s]

network parameters: DeepCut_resnet50_Cow_labelingJan2shuffle1_21000
Fitting state-space models with parameters 3 1


100%|██████████| 4/4 [00:01<00:00,  1.58it/s]


Method  fitting  found  201  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, ARdegree, MAdegree, alpha, comparisonbodyparts) or use a different method.
yes/noyes


37it [00:00, 369.97it/s]

Frames from video cow1  already extracted (more will be added)!
Loading video...
Duration of video [s]:  10.05 , recorded @  20.0 fps!
Overall # of frames:  201 with (cropped) frame dimensions:  [680, 420]
Kmeans-quantization based extracting of frames from 0.0  seconds to 10.05  seconds.
Extracting... 200  (this might take a while).


200it [00:00, 387.69it/s]


Kmeans clustering...(this might take a while).
Let's select frames indices: [173, 129, 61, 80, 110, 199, 17, 146, 30, 102, 165, 47, 77, 56, 6, 116, 157, 26, 183, 137]
AUTOMATIC ADDING OF VIDEO TO CONFIG FILE FAILED! You need to do this manually for including it in the config.yaml file!
Videopath: /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1.mp4 Coordinates for cropping: None
The outlier frames are extracted. They are stored in the subdirectory labeled-data\cow1.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [10]:
%gui wx
deeplabcut.refine_labels(path_config_file)

ERROR:root:Cannot activate multiple GUI eventloops


/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/labeled-data/cow1
Checking labels if they are outside the image
Found head outside the image labeled-data/cow1/img199.png.Setting it to NaN
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [12]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

Merged data sets and updated refinement iteration to 1.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [13]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [14]:
#Perhaps plot the labels to see how how all the frames are annoted (including the refined ones)
deeplabcut.check_labels(path_config_file)

Creating images with labels by FKIE.
/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/labeled-data/cow1_labeled  already exists!
They are stored in the following folder: /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/labeled-data/cow1_labeled.
/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/labeled-data/cow24_labeled  already exists!
They are stored in the following folder: /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/labeled-data/cow24_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


Now one can train the network again... (with the expanded data set)

In [ ]:
deeplabcut.train_network(path_config_file, gputouse=0)

INFO:tensorflow:Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-20000


INFO:tensorflow:Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-20000
Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-20000
Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-20000
Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-20000
Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-0/Cow_labelingJan2-trainset90shuffle1/train/snapshot-20000
Restoring parameters from /home/wei-chan/Documents/Thesis/src/annotation/C

Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/wei-chan/Documents/Thesis/src/annotation/Cow_labeling-FKIE-2019-01-02/dlc-models/iteration-1/Cow_labelingJan2-trainset90shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight': 400, 'all_joints': [[0], [1], [2], [3], [4], [5], [6]

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [13]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)


  1%|▏         | 3/201 [00:00<00:08, 24.45it/s]

Starting %  /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4 ['/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1.mp4', '/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow24.mp4']
Loading  /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1.mp4 and data.
False 0 680 0 420
201
Duration of video [s]:  10.05 , recorded with  20.0 fps!
Overall # of frames:  201 with cropped frame dimensions:  680 420
Generating frames and creating video.



  2%|▏         | 3/157 [00:00<00:06, 24.94it/s]

Starting %  /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4 ['/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow1.mp4', '/home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow24.mp4']
Loading  /home/wei-chan/Documents/Thesis/src/annotation/data/Tag4/cow24.mp4 and data.
False 0 680 0 420
157
Duration of video [s]:  7.85 , recorded with  20.0 fps!
Overall # of frames:  157 with cropped frame dimensions:  680 420
Generating frames and creating video.



100%|██████████| 157/157 [00:05<00:00, 29.47it/s]

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [0]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)